In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
_FOLDER = 'drive/MyDrive/A2_Data/T3_Data/images/'
_TRAIN_ANGRY_FOLDER = _FOLDER + 'train/angry'
_TRAIN_DISGUST_FOLDER = _FOLDER + 'train/disgust'
_TRAIN_FEAR_FOLDER = _FOLDER + 'train/fear'
_TRAIN_HAPPY_FOLDER = _FOLDER + 'train/happy'
_TRAIN_NEUTRAL_FOLDER = _FOLDER + 'train/neutral'
_TRAIN_SAD_FOLDER = _FOLDER + 'train/sad'
_TRAIN_SURPRISE_FOLDER = _FOLDER + 'train/surprise'

_VALIDATION_ANGRY_FOLDER = _FOLDER + 'validation/angry'
_VALIDATION_DISGUST_FOLDER = _FOLDER + 'validation/disgust'
_VALIDATION_FEAR_FOLDER = _FOLDER + 'validation/fear'
_VALIDATION_HAPPY_FOLDER = _FOLDER + 'validation/happy'
_VALIDATION_NEUTRAL_FOLDER = _FOLDER + 'validation/neutral'
_VALIDATION_SAD_FOLDER = _FOLDER + 'validation/sad'
_VALIDATION_SURPRISE_FOLDER = _FOLDER + 'validation/surprise'

_TRAINING_DIRECTORIES_LABELS = [
  (_TRAIN_ANGRY_FOLDER,'angry'),
  (_TRAIN_DISGUST_FOLDER,'disgust'),
  (_TRAIN_FEAR_FOLDER,'fear'),
  (_TRAIN_HAPPY_FOLDER,'happy'),
  (_TRAIN_NEUTRAL_FOLDER,'neutral'),
  (_TRAIN_SAD_FOLDER,'sad'),
  (_TRAIN_SURPRISE_FOLDER,'surprise')
]

_VALIDATION_DIRECTORIES_LABELS = [
  (_VALIDATION_ANGRY_FOLDER,'angry'),
  (_VALIDATION_DISGUST_FOLDER,'disgust'),
  (_VALIDATION_FEAR_FOLDER,'fear'),
  (_VALIDATION_HAPPY_FOLDER,'happy'),
  (_VALIDATION_NEUTRAL_FOLDER,'neutral'),
  (_VALIDATION_SAD_FOLDER,'sad'),
  (_VALIDATION_SURPRISE_FOLDER,'surprise')
]

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import random

_PICK_BATCH_SIZE = 5000
_CLASSIFICATION_SIZE = 7

def one_hot_encode(x,size=_CLASSIFICATION_SIZE):
  label = {'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4,'sad':5,'surprise':6}
  x = label[x]
  ohe = np.zeros(size)
  ohe[x]=1
  return ohe

training_images_labels_list_all = []
for training_directory_label in _TRAINING_DIRECTORIES_LABELS:
  label = training_directory_label[1]
  directory = training_directory_label[0]
  print('\n',directory)
  for filename in os.listdir(directory):
      if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
        if(len(training_images_labels_list_all)%100==0):
          print('.',end='')
        image = cv2.imread(os.path.join(directory, filename))
        training_images_labels_list_all.append([image,label])
      else:
          print(f'Unable to read image: {filename}')

print(f'Total images in the training set: {len(training_images_labels_list_all)}')


 drive/MyDrive/A2_Data/T3_Data/images/train/angry
........................................
 drive/MyDrive/A2_Data/T3_Data/images/train/disgust
.....
 drive/MyDrive/A2_Data/T3_Data/images/train/fear
.........................................
 drive/MyDrive/A2_Data/T3_Data/images/train/happy
.......................................................................
 drive/MyDrive/A2_Data/T3_Data/images/train/neutral
..................................................
 drive/MyDrive/A2_Data/T3_Data/images/train/sad
..................................................
 drive/MyDrive/A2_Data/T3_Data/images/train/surprise
................................Total images in the training set: 28823


In [ ]:
#test train split
def get_col(array,index):
  arr = []
  for row in array:
    arr.append(row[index])
  return arr

def encode_labels(y):
  arr = []
  for label in y:
    arr.append(one_hot_encode(label))
  return arr

from sklearn.model_selection import train_test_split
X = get_col(training_images_labels_list_all,0)
y = get_col(training_images_labels_list_all,1)
y = encode_labels(y)
X_train_imgs_locations, X_test_img_locations, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train_imgs_locations), len(y_train), len(X_test_img_locations), len(y_test)

(23058, 23058, 5765, 5765)

In [ ]:
#config model
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=14336)]) # Notice here
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


#declare model
history = []
model = None
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

_IMAGES_SHAPE = (48, 48, 3)
epochs = 20
batch_size = 10
model = Sequential([
    Conv2D(2, (2, 2), activation='relu', input_shape=_IMAGES_SHAPE),
    MaxPooling2D((2, 2)),
    Conv2D(4, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(8, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.6),
    Dense(7, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#pick a number of images from the training set to train model on
train_images_list = X_train_imgs_locations
train_labels_list = y_train


In [ ]:
# Split dataset
X_train, X_val, y_train, y_val = train_test_split(train_images_list, train_labels_list, test_size=0.2, random_state=42)
np.shape(X_train), np.shape(y_train), np.shape(X_val), np.shape(y_val)

((18446, 48, 48, 3), (18446, 7), (4612, 48, 48, 3), (4612, 7))

In [ ]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1
)
datagen.fit(X_train)
_IMAGES_SHAPE = (48, 48, 3)
# Model architecture
model = Sequential([
    Conv2D(128, (2, 2), activation='relu', input_shape=_IMAGES_SHAPE),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.5),
    Conv2D(128, (5, 5), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(512, (3, 3), activation='relu'),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(_CLASSIFICATION_SIZE, activation='softmax')
])

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=1,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 48

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

print(np.shape(X_train), np.shape(y_train), np.shape(X_val), np.shape(y_val))
# Train the model
with tf.device('/device:GPU:0'):
  history = model.fit(np.array(X_train), np.array(y_train),  batch_size=42, validation_data=(np.array(X_val), np.array(y_val)), epochs=20)

(18446, 48, 48, 3) (18446, 7) (4612, 48, 48, 3) (4612, 7)
Epoch 1/20
440/440 [==============================] - 14s 24ms/step - loss: 1.6927 - accuracy: 0.3460 - val_loss: 1.5162 - val_accuracy: 0.4198
Epoch 2/20
440/440 [==============================] - 10s 23ms/step - loss: 1.4496 - accuracy: 0.4373 - val_loss: 1.3808 - val_accuracy: 0.4699
Epoch 3/20
440/440 [==============================] - 11s 24ms/step - loss: 1.3483 - accuracy: 0.4829 - val_loss: 1.5234 - val_accuracy: 0.3990
Epoch 4/20
440/440 [==============================] - 10s 23ms/step - loss: 1.2713 - accuracy: 0.5119 - val_loss: 1.4191 - val_accuracy: 0.4525
Epoch 5/20
440/440 [==============================] - 10s 23ms/step - loss: 1.1846 - accuracy: 0.5491 - val_loss: 1.3868 - val_accuracy: 0.4872
Epoch 6/20
440/440 [==============================] - 10s 23ms/step - loss: 1.0930 - accuracy: 0.5856 - val_loss: 1.3261 - val_accuracy: 0.5104
Epoch 7/20
440/440 [==============================] - 10s 23ms/step - loss: 0.

In [ ]:
from tensorflow.keras.models import save_model

# Define the file path where you want to save the model
model_path = 'facial_expression_recognission_3.0.h5'

# Save the model to the specified file path
model.save(model_path)

print(f"Model saved to {model_path}")

Model saved to facial_expression_recognission_3.0.h5


In [ ]:
print(history.history)

{'loss': [1.6927040815353394, 1.4496334791183472, 1.348257303237915, 1.271256446838379, 1.1846392154693604, 1.093000888824463, 0.9888171553611755, 0.8836280703544617, 0.769575297832489, 0.6670976281166077, 0.5790687203407288, 0.49061349034309387, 0.4350476562976837, 0.37256547808647156, 0.3314194679260254, 0.2829115092754364, 0.2691361904144287, 0.24668434262275696, 0.2194606214761734, 0.19655077159404755], 'accuracy': [0.3460370898246765, 0.43727636337280273, 0.48292312026023865, 0.5119267106056213, 0.5490621328353882, 0.5856012105941772, 0.6283746957778931, 0.6721240282058716, 0.7139217257499695, 0.7555025219917297, 0.7896562814712524, 0.8235389590263367, 0.8456575870513916, 0.8670714497566223, 0.8860999941825867, 0.9014962315559387, 0.9076764583587646, 0.9198742508888245, 0.9263255000114441, 0.9352705478668213], 'val_loss': [1.5162453651428223, 1.380768060684204, 1.5234134197235107, 1.4191285371780396, 1.3868173360824585, 1.326119303703308, 1.3430826663970947, 1.7300190925598145, 1.

In [44]:
file_path = '3.0'+'_history.txt'
with open(file_path, 'w') as file:
    file.write(str(history.history))

In [ ]:
X_test = X_test_img_locations
y_test = y_test

In [ ]:
np.shape(X_test)

(5765, 48, 48, 3)

In [ ]:
predicted = model.predict(np.array(X_test))

181/181 [==============================] - 1s 5ms/step


In [ ]:
print(y_test[0])

[0. 0. 0. 1. 0. 0. 0.]


In [ ]:
def get_index_of_max_ele(arr):
  max_index = 0  # Initialize the index of the maximum element to 0
  max_element = arr[0]  # Initialize the maximum element to the first element

  for i in range(1, len(arr)):
      if arr[i] > max_element:
          max_element = arr[i]
          max_index = i

  return max_index
def remap(predicted):
  arr = []
  for i in predicted:
    arr.append(get_index_of_max_ele(i))
  return arr

def get_accuracy(predicted,actual):
  accuracy = 0
  for i in range(len(predicted)):
    if(predicted[i]==actual[i]):
      accuracy+=1
  return accuracy/len(predicted)

accuracy_on_test = get_accuracy(remap(predicted),remap(y_test))

In [ ]:
print(accuracy_on_test)

0.5257588898525586
